# DeepSeek Agent 实战：小红书爆款文案生成助手

本 Notebook 将指导您如何使用 DeepSeek LLM 构建一个能够生成小红书爆款文案的智能 Agent。我们将从需求拆解开始，逐步定义 Agent 的系统提示词 (System Prompt)、外部工具 (Tools)，并实现其核心的工作流程，最终生成符合小红书平台特点的文案。

## 1. 环境准备与DeepSeek API配置

In [1]:
!pip install pandas

In [2]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0 sentence-transformers==3.0.1

In [3]:
import os
from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

# 初始化 DeepSeek 客户端

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

## 2. 需求拆解与Agent任务规划

#### 用户痛点与核心需求：
*   **效率低下：** 人工创作周期长，难以满足高频发布需求。
*   **创意瓶颈：** 难以持续产出新颖、吸引人的爆款创意。
*   **趋势捕捉难：** 实时流行元素难以快速融入文案。
*   **平台特性把握：** 小红书特有风格（标题、正文、标签、表情）难以精准复制。

#### “爆款”文案的特征：
*   **强吸引力标题：** 制造好奇、痛点共鸣、利益点清晰。
*   **沉浸式正文：** 真实体验分享、细节描述、情感共鸣。
*   **精准且多样标签：** 热门话题、品牌词、产品词、垂直领域词。
*   **生动表情符号：** 增强表达力，提升活泼感。
*   **清晰的行动召唤 (CTA)。**

#### Agent 任务规划：核心工作流
1.  **用户指令接收：** 接收产品信息、主题、风格等。
2.  **信息收集 (Web Search/DB Query)：** 实时搜索行业趋势、热门话题、竞品分析、产品卖点。
3.  **内容构思与初稿生成 (LLM)：** 结合所有信息，撰写标题、正文、标签、表情符号。
4.  **风格与格式优化 (LLM)：** 根据小红书平台特点和指定风格，对文案进行润色和结构调整。
5.  **最终输出：** 呈现完整文案。

## 3. 爆款文案生成逻辑与 Prompt 设计

### 3.1 System Prompt (系统提示词)

System Prompt 是 Agent 的“大脑”和“行为准则”。它定义了 Agent 的角色、目标以及工作方式。我们将采用 `Thought-Action-Observation` (ReAct) 模式来引导 DeepSeek 的推理过程。

In [4]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

### 3.2 Tools (工具定义)

Agent 的“双手”由一系列可调用的工具组成。这些工具扩展了 LLM 的能力，使其能够获取实时信息、查询数据库或执行特定操作。在这里，我们定义了三个核心工具：

*   `search_web`: 用于搜索互联网上的实时信息，如最新趋势、用户评价等。
*   `query_product_database`: 用于查询产品数据库，获取产品的详细卖点和特点。**此工具为模拟**。
*   `generate_emoji`: 用于根据文案内容生成恰当的表情符号。**此工具为模拟**。

In [5]:
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的关键词或问题，例如'最新小红书美妆趋势'或'深海蓝藻保湿面膜 用户评价'"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的产品名称，例如'深海蓝藻保湿面膜'"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

### 3.3 模拟工具实现

由于我们无法直接调用真实的外部 API (如Google Search或内部产品数据库)，我们将创建一些模拟 (Mock) 工具函数来演示 Agent 的工作流程。在实际应用中，您需要将这些模拟函数替换为真实的 API 调用。

In [6]:
from glob import glob

text_lines = []

for file_path in glob("products.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [7]:
len(text_lines)

51

In [8]:
from pymilvus import model as milvus_model

# OpenAI国内代理 https://api.apiyi.com/token 
embedding_model = milvus_model.dense.OpenAIEmbeddingFunction(
    model_name='text-embedding-3-large', # Specify the model name
    api_key='sk-GWtxYBmrvJW0JiHtDf745f1582Db489a82BbEcD59e279538', # Provide your OpenAI API key
    base_url='https://api.apiyi.com/v1',
    dimensions=512
)

/Users/wjx/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

512
[-0.02817117  0.00430343 -0.01862586  0.08186801 -0.03163895 -0.05273157
 -0.04879905  0.12469677 -0.02091388  0.03979   ]


In [10]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_rednote.db")

collection_name = "products_rag_collection"

/Users/wjx/miniconda3/lib/python3.13/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [11]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [12]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

In [13]:
from tqdm import tqdm

# 1. 清洗 text_lines，移除空文本和过长的文本
clean_text_lines = []
for i, t in enumerate(text_lines):
    if not isinstance(t, str):
        continue
    t = t.strip()
    if not t:  # 跳过空文本
        continue
    # 限制文本长度，防止超过 API 限制（8000 字符）
    if len(t) > 8000:
        t = t[:8000]
    clean_text_lines.append(t)

print(f"原始段落数: {len(text_lines)}")
print(f"清洗后段落数: {len(clean_text_lines)}")

# 2. 生成向量嵌入
data = []
doc_embeddings = embedding_model.encode_documents(clean_text_lines)

# 3. 准备插入数据
for i, line in enumerate(tqdm(clean_text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

# 4. 插入到 Milvus
milvus_client.insert(collection_name=collection_name, data=data)
print(f"✅ 成功插入 {len(data)} 条产品数据到 Milvus")

原始段落数: 51
清洗后段落数: 50


Creating embeddings: 100%|████| 50/50 [00:00<00:00, 105970.29it/s]

✅ 成功插入 50 条产品数据到 Milvus


## 4. 实战：构建小红书文案生成 Agent

现在，我们将把 System Prompt、工具定义和模拟工具函数整合起来，构建出能够自动执行的 DeepSeek Agent 工作流。核心是 `generate_rednote` 函数，它通过一个循环来模拟 Agent 的 `Thought-Action-Observation` 过程。

In [15]:
import json
import re
from openai import OpenAI
import os

# ===== 初始化客户端 =====
# 使用环境变量中的 DeepSeek API Key（已在 Cell 3 中设置）
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

# 注意：Milvus 向量数据库和 embedding_model 已在之前的 Cell 中初始化
# - embedding_model: Cell 12
# - milvus_client: Cell 14
# - collection_name: Cell 14
# - 数据已插入: Cell 17

# ===== 系统提示词 =====

SYSTEM_PROMPT = """
你是一个专业的小红书爆款文案生成助手。你的任务是：

1. 理解产品信息：根据用户提供的产品名称，使用工具查询详细的产品信息
2. 分析产品特点：深入理解产品的成分、功效、适用人群等
3. 生成爆款文案：创作吸引眼球、能激发购买欲的小红书文案

文案要求：
- 【标题】: 简洁有力，包含产品核心卖点，长度8-20字
- 【正文】: 生动有趣，分3-4段，共150-300字，讲述产品故事和用户收益
- 【标签】: 至少5个相关标签，用#开头，容易被搜索
- 【表情】: 至少5个相关emoji，恰当点缀在文案中

工作流程：
1. 首先使用 query_product_database 工具查询产品信息
2. 分析查询结果中的成分、功效、适合肤质等关键信息
3. 基于产品信息，结合用户指定的风格，生成创意文案
4. 确保输出是完整的JSON格式

重要提示：
- 在生成最终文案前，必须先查询产品信息
- 文案要突出产品的独特卖点和用户痛点解决方案
- 使用指定的语气风格（活泼甜美/知性/搞怪等）
- 确保JSON格式正确，用```json```包裹
"""

# ===== 工具定义（改为RAG查询） =====

TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "从产品向量数据库查询产品的详细信息，包括成分、功效、适合肤质、价格等。这是生成文案前必须执行的第一步。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的产品名称，例如'深海蓝藻保湿面膜'、'烟酰胺美白精华'等"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_product_features",
            "description": "分析产品的核心特点和卖点，基于产品信息提取关键信息用于文案创作",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_info": {
                        "type": "string",
                        "description": "产品的详细信息文本（从query_product_database获取）"
                    }
                },
                "required": ["product_info"]
            }
        }
    }
]

# ===== 工具实现函数 =====

def query_product_database(product_name: str) -> str:
    """
    从Milvus向量数据库查询产品信息（基于RAG实现）
    
    参考 DeepSeek RAG 课程中的方法，使用向量相似度搜索来查找相关产品信息。
    
    Args:
        product_name: 产品名称
        
    Returns:
        格式化的产品信息字符串
    """
    print(f"[RAG工具] 查询产品: {product_name}")
    
    try:
        # 1. 将产品名称转换为向量嵌入
        query_embeddings = embedding_model.encode_queries([product_name])
        
        # 2. 在Milvus collection中搜索最相关的产品信息
        # limit=3 表示返回前3个最相关的结果
        search_results = milvus_client.search(
            collection_name=collection_name,
            data=query_embeddings,  # 将查询转换为嵌入向量
            limit=3,  # 返回前3个最相关的结果
            search_params={"metric_type": "IP", "params": {}},  # 使用内积距离
            output_fields=["text"],  # 返回 text 字段
        )
        
        # 3. 提取检索到的文档文本
        if search_results and len(search_results) > 0 and len(search_results[0]) > 0:
            retrieved_docs = []
            for result in search_results[0]:
                if "entity" in result and "text" in result["entity"]:
                    retrieved_docs.append(result["entity"]["text"])
            
            # 4. 将检索到的文档合并为上下文
            if retrieved_docs:
                context = "\n\n".join(retrieved_docs)
                print(f"[RAG工具] 查询成功，找到 {len(retrieved_docs)} 条相关产品信息")
                return context
            else:
                return f"未找到关于 '{product_name}' 的产品信息。"
        else:
            return f"未找到关于 '{product_name}' 的产品信息。"
        
    except Exception as e:
        error_msg = f"[RAG工具] 查询失败: {str(e)}"
        print(error_msg)
        return error_msg

def analyze_product_features(product_info: str) -> str:
    """
    分析产品的核心特点
    
    Args:
        product_info: 产品详细信息
        
    Returns:
        分析结果（包含卖点、适合人群等）
    """
    print("[分析工具] 分析产品特点...")
    
    # 提取关键信息
    analysis = {
        "卖点": [],
        "主要成分": [],
        "适合人群": [],
        "用户痛点解决": []
    }
    
    # 简单的文本分析（实际可以用AI分析）
    if "补水" in product_info or "保湿" in product_info:
        analysis["卖点"].append("强效补水保湿")
        analysis["用户痛点解决"].append("解决肌肤干燥问题")
    
    if "美白" in product_info:
        analysis["卖点"].append("美白提亮")
        analysis["用户痛点解决"].append("改善肤色暗沉")
    
    if "抗衰" in product_info or "细纹" in product_info:
        analysis["卖点"].append("抗衰紧致")
        analysis["用户痛点解决"].append("淡化细纹")
    
    if "敏感" in product_info or "舒缓" in product_info:
        analysis["卖点"].append("温和舒缓")
        analysis["用户痛点解决"].append("适合敏感肌")
    
    result = json.dumps(analysis, ensure_ascii=False)
    print(f"[分析工具] 分析完成: {result}")
    return result

# ===== 工具映射 =====

available_tools = {
    "query_product_database": query_product_database,
    "analyze_product_features": analyze_product_features,
}

# ===== 主函数：Agent文案生成 =====

def generate_rednote(
    product_name: str, 
    tone_style: str = "活泼甜美", 
    max_iterations: int = 8
) -> str:
    """
    使用 DeepSeek Agent + Milvus RAG 生成小红书爆款文案。
    
    工作流程：
    1. 用户输入产品名称
    2. Agent调用query_product_database查询向量库中的产品信息
    3. Agent调用analyze_product_features分析产品特点
    4. Agent基于产品信息和分析结果，生成创意文案
    5. 返回JSON格式的文案
    
    Args:
        product_name (str): 产品名称，例如"深海蓝藻保湿面膜"
        tone_style (str): 文案风格，如"活泼甜美"、"知性"、"搞怪"等
        max_iterations (int): 最大迭代次数，防止无限循环
        
    Returns:
        str: 生成的爆款文案（JSON格式字符串）
    """
    
    print(f"\n{'='*70}")
    print(f"🚀 启动小红书爆款文案生成器")
    print(f"📦 产品: {product_name}")
    print(f"✨ 风格: {tone_style}")
    print(f"{'='*70}\n")
    
    # 对话历史
    messages = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"""请为产品「{product_name}」生成一篇小红书爆款文案。

要求：
1. 首先查询产品数据库获取产品详细信息
2. 分析产品的核心特点和卖点
3. 基于分析结果，用{tone_style}的语气生成文案
4. 文案包含：标题（8-20字）、正文（150-300字，分3-4段）、至少5个标签、至少5个emoji
5. 输出完整的JSON格式，用```json{{...}}```包裹

JSON格式示例：
```json
{{
    "title": "文案标题",
    "content": "文案正文内容",
    "tags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
    "emojis": ["emoji1", "emoji2", "emoji3", "emoji4", "emoji5"],
    "style": "{tone_style}"
}}
```
"""
        }
    ]
    
    iteration_count = 0
    
    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"\n{'─'*70}")
        print(f"⏭️  迭代 #{iteration_count}")
        print(f"{'─'*70}")
        
        try:
            # 调用DeepSeek API
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=TOOLS_DEFINITION,
                tool_choice="auto"
            )
            
            response_message = response.choices[0].message
            
            # ===== 处理工具调用 =====
            if response_message.tool_calls:
                print(f"🔧 Agent决定调用工具...")
                messages.append(response_message)
                
                tool_outputs = []
                
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}
                    
                    print(f"\n   📌 工具: {function_name}")
                    print(f"   📝 参数: {function_args}")
                    
                    # 执行工具
                    if function_name in available_tools:
                        tool_function = available_tools[function_name]
                        tool_result = tool_function(**function_args)
                        print(f"   ✅ 工具返回结果 (前200字):")
                        print(f"      {str(tool_result)[:200]}...")
                        
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": str(tool_result)
                        })
                    else:
                        error_msg = f"❌ 未知工具: {function_name}"
                        print(f"   {error_msg}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": error_msg
                        })
                
                messages.extend(tool_outputs)
                print(f"\n   将工具结果添加到对话历史，继续Agent推理...")
            
            # ===== 处理最终响应 =====
            elif response_message.content:
                print(f"\n📄 Agent生成内容:")
                print(f"   {response_message.content[:300]}...")
                
                # 尝试提取JSON
                json_match = re.search(
                    r"```json\s*(\{.*?\})\s*```",
                    response_message.content,
                    re.DOTALL
                )
                
                if json_match:
                    try:
                        json_str = json_match.group(1)
                        final_response = json.loads(json_str)
                        
                        print(f"\n{'='*70}")
                        print(f"✨ 文案生成成功！")
                        print(f"{'='*70}\n")
                        
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                        
                    except json.JSONDecodeError as e:
                        print(f"   ⚠️ JSON解析失败: {e}")
                        messages.append(response_message)
                
                else:
                    # 直接尝试解析整个content
                    try:
                        final_response = json.loads(response_message.content)
                        print(f"\n{'='*70}")
                        print(f"✨ 文案生成成功！")
                        print(f"{'='*70}\n")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        print(f"   ⚠️ 内容非JSON格式，继续交互...")
                        messages.append(response_message)
            
            else:
                print("   ⚠️ 未知响应类型")
                break
        
        except Exception as e:
            print(f"   ❌ API调用错误: {e}")
            break
    
    print(f"\n{'='*70}")
    print(f"⚠️ 达到最大迭代次数或生成失败")
    print(f"{'='*70}\n")
    return "❌ 未能成功生成文案"

## 5. 实际测试与文案生成

现在，让我们调用我们构建的 `generate_rednote` 函数，看看它能生成什么样的爆款文案！

In [16]:
# 测试案例 1: 小红书美妆趋势
product_name_1 = "身体美白乳液"
tone_style_1 = "知性温柔"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书爆款文案生成器
📦 产品: 身体美白乳液
✨ 风格: 知性温柔


──────────────────────────────────────────────────────────────────────
⏭️  迭代 #1
──────────────────────────────────────────────────────────────────────
🔧 Agent决定调用工具...

   📌 工具: query_product_database
   📝 参数: {'product_name': '身体美白乳液'}
[RAG工具] 查询产品: 身体美白乳液
[RAG工具] 查询成功，找到 3 条相关产品信息
   ✅ 工具返回结果 (前200字):
      身体美白乳液

**品牌**: 身体护肤  
**价格**: ¥129  
**评分**: 4.6⭐ (5430条评价)  
**规格**: 200ml  
**产地**: 韩国  
**库存**: 3240件  
**保质期**: 2年  

##

产品描述
全身美白乳液采用烟酰胺3%+光甘草定+视黄醇棕榈酸酯组合配方，改善身体肤色不均，淡化痘印，4周明显提亮，实现全身美白。

##

主要功...

   将工具结果添加到对话历史，继续Agent推理...

──────────────────────────────────────────────────────────────────────
⏭️  迭代 #2
──────────────────────────────────────────────────────────────────────
🔧 Agent决定调用工具...

   📌 工具: analyze_product_features
   📝 参数: {'product_info': '身体美白乳液\n\n**品牌**: 身体护肤  \n**价格**: ¥129  \n**评分**: 4.6⭐ (5430条评价)  \n**规格**: 200ml  \n**产地**: 韩国  \n**库存**: 3240件  \n**保质期**: 2年  \n\n##\n\n产品描述\n全身美白乳液采用烟酰胺3%+光甘草定+视黄醇棕榈酸酯组合配方，改善身体

### 格式化 小红书文案

**格式化函数 `format_rednote_for_markdown` 的功能：**

1. 解析 JSON 字符串。
2. 提取标题、正文、标签和表情符号。
3. 将它们组合成一个易读的 Markdown 格式的文本。


**工作方式：**

1. **解析 JSON**：使用 `json.loads()` 将输入的字符串转换为 Python 字典。如果解析失败，会返回一个错误信息。
2. **提取数据**：使用 `.get()` 方法从字典中安全地提取 `title`、`body` 和 `hashtags`。使用 `.get()` 的好处是，如果某个键不存在，它会返回一个默认值（例如 `None` 或空列表），而不是抛出 `KeyError`。
3. **构建 Markdown 标题**：将 `title` 格式化为 Markdown 的二级标题 (`## Title`)。
4. **处理正文**：直接使用 `body`。由于小红书正文中的换行很重要，我们保留它们。
5. **处理 Hashtags**：将 `hashtags` 列表中的每个标签用空格连接起来，形成一行。
6. **表情符号 (Emojis)**：在小红书的实际发布中，表情符号通常已经嵌入在标题和正文中了。这个函数没有单独列出它们，因为这通常不是最终发布格式的一部分。如果需要，可以取消注释相关代码来单独显示它们。
7. **返回结果**：返回拼接好的 Markdown 字符串，并使用 `.strip()` 去除可能存在于末尾的多余空白。

In [28]:
# 重新定义格式化函数
import json

def format_rednote_for_markdown(json_string: str) -> str:
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    # 支持两种字段名：content 或 body
    body = data.get("content", data.get("body", ""))
    # 支持两种字段名：tags 或 hashtags
    hashtags = data.get("tags", data.get("hashtags", []))
    
    markdown_output = f"## {title}\n\n"
    
    if body:
        # 将 \\n 转换为实际的换行符
        body = body.replace("\\n", "\n")
        markdown_output += f"{body}\n\n"
    
    if hashtags:
        hashtag_string = " ".join(hashtags)
        markdown_output += f"{hashtag_string}\n"
        
    return markdown_output.strip()

In [29]:
# 调用格式化函数
markdown_note_1 = format_rednote_for_markdown(result_1)
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note_1)

--- 格式化后的小红书文案 (Markdown) ---
## 身体美白新体验：4周见证全身焕亮✨

🌿 最近发现了一款让我惊喜的身体美白乳液，它不仅仅是简单的保湿，而是真正能改善肤色的宝藏产品。作为一个注重全身护理的人，我一直在寻找既能美白又能滋养肌肤的乳液，这款产品完全满足了我的需求。

💫 它的核心配方真的很用心——3%烟酰胺+光甘草定+视黄醇棕榈酸酯，三重美白成分协同作用。烟酰胺抑制黑色素转移，光甘草定从源头阻断黑色素生成，视黄醇促进角质更新，这样的组合让美白效果更加全面持久。

🌸 使用感受也很棒，乳液质地轻盈不粘腻，吸收很快，涂抹后肌肤立刻变得水润有光泽。坚持使用4周后，我发现身体肤色明显均匀了许多，特别是膝盖、手肘这些容易暗沉的部位改善很明显。

✨ 最让我满意的是它还能淡化身体上的小痘印和疤痕，让肌肤整体看起来更加光滑细腻。现在夏天穿裙子、短裤都更有自信了，全身肌肤都散发着健康的光泽感。

#身体美白 #全身护理 #烟酰胺美白 #肤色均匀 #身体乳液推荐 #韩国护肤 #夏日美白


## 6. 评估与优化

文案生成并非一蹴而就，需要持续的评估和优化。本节讨论一些评估方法和优化策略。

#### 评估文案质量：
*   **客观量化评估 (数据)：**
    *   **点赞/收藏/评论/分享：** 基础互动
    *   **曝光/阅读/点击/涨粉：：** 流量与曝光
    *   **停留时长/截图率：** 用户行为。
    *   **商品页浏览/加购/ROI/成交转化：** 商业价值
    *   **爆文率/同类横向对比：** 竞争对比
*   **主观内部评估 (人工)：**
    *   **相关性：** 是否符合产品特点和主题。
    *   **吸引力：** 标题是否抓人，内容是否流畅。
    *   **合规性：** 是否有敏感词、违规宣传。
    *   **风格匹配：** 是否符合小红书调性和指定语气。
    *   **用户画像：** 目标人群年龄、地域、兴趣标签。



#### 优化迭代方法：
*   **Prompt 调整：** 根据评估结果，精修 System Prompt、User Prompt，增加或修改 Few-shot 示例。
*   **工具扩充：** 引入新的工具（如敏感词检测工具、竞品分析工具）。
*   **RAG (检索增强生成)：** 结合更精准的内部知识库，减少幻觉。


## 7. 总结与展望

通过本次实战，我们成功构建了一个基于 DeepSeek Agent 的小红书爆款文案生成助手。我们学习了如何拆解需求、设计 Prompt、定义工具，并实现 Agent 的核心工作流。

Agent 在内容营销领域的潜力巨大，未来可以进一步拓展到：

*   **超个性化内容：** 根据用户数据，生成一对一的定制文案。
*   **多模态内容创作：** 结合图片、视频生成，实现图文音视频一体化。
*   **智能营销决策：** Agent 不仅生成内容，还能分析效果并给出投放建议。
*   **跨平台适配：** 快速生成适应不同社交媒体平台风格的文案。

同时，我们也需关注挑战，如确保内容真实性、处理高度主观情感、与现有工作流的无缝集成等。Agent 技术仍在快速发展，期待未来能带来更多惊喜！